In [83]:
import pandas as pd

In [84]:
df = pd.read_csv('MSHUB_feature_table_Metadata_combined_blank_subtracted_sample_vs_blank_for_MA.csv')
df.head()

,filename,ATTRIBUTE_sample,ATTRIBUTE_Source,ATTRIBUTE_GasProduction24hmLg,ATTRIBUTE_GPcat,ATTRIBUTE_CH4ppm,ATTRIBUTE_CH424hmLg,ATTRIBUTE_CH4Ycat,ATTRIBUTE_TVFAcat,ATTRIBUTE_Acecat,...,1041_0.37,1043_0.37,1044_0.37,1045_0.37,1049_0.37,1050_0.37,1051_0.37,1052_0.37,1053_0.37,1054_0.37
0,1.mzML,Sample,Asparagopsis taxiformis,114.7,low,82.0,0.11,low,low,low,...,175.739959,7439.671710,301.370004,2461.011948,11726.78016,23.012886,0.0000,30.300921,0.000000,0.0
1,10.mzML,Sample,Mastocarpus papillatus,130.5,norm,6825.0,10.65,norm,norm,norm,...,0.000000,0.000000,473.284180,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.0
2,11.mzML,Sample,Sargassum fluitans,131.4,norm,6751.0,10.60,norm,norm,norm,...,0.000000,25.133266,3.224939,46.901519,0.00000,0.000000,87787.0909,0.000000,88.535568,0.0
3,12.mzML,Sample,Dictyopteris undulata,129.5,norm,7625.0,11.84,norm,norm,norm,...,17989.395940,1384.704414,1980.594277,0.000000,0.00000,136.514312,0.0000,21.122395,202.982616,0.0
4,13.mzML,Sample,Soleria tenera,126.4,norm,5849.0,13.86,norm,norm,norm,...,0.000000,1115.182126,213.789898,15.078425,0.00000,0.000000,0.0000,0.000000,0.000000,0.0


In [85]:
def filterBlank(feat_table, prop_blank_feats=1.0, prop_samples=0.9):
    """Filters out features that have intensity less than a desired
       proportion of intensity in blank samples at a given
       number of samples
    Parameters
    ----------
    feat_table: pd.DataFrame
        DataFrame containing columns with ATTRIBUTE_ suffix.
        Blank samples should contain "B(b)lank" in the name
    prop_blank_feats: float
        Proportion of blank intensitie that a feature has to be greater to be keept.
    prop_samples: float
        Proportion of samples in which the feature has to be greated than blank average proportion.
    Returns
        Filtered pd.DataFrame.
    -------
    """
    last_attr = feat_table.columns[feat_table.columns.str.contains('ATTRIBUTE')][-1]
    plast_attr = feat_table.columns.get_loc(last_attr)+1
    mask = feat_table.filename.str.lower().str.contains('blank')
    print('Found %s samples containing "blank" in the name.' % mask.sum())
    print('Found %s features.' % (feat_table.shape[1]-plast_attr))
    blank_mean = feat_table.loc[mask, feat_table.columns[plast_attr:]].mean()
    feat_less_blank = (feat_table.loc[~mask, feat_table.columns[plast_attr:]]>(prop_blank_feats*blank_mean)).sum()
    prop_feat_less_blank = feat_less_blank > prop_samples*(feat_table.shape[0]-mask.sum())
    nprop = prop_feat_less_blank.sum()
    print(f'''Found {nprop} features whose intensity was greater than {prop_blank_feats*100}% of the
          average intensity of blank samples in {prop_samples*100}% samples at least.''')
    return pd.concat([feat_table.loc[~mask, feat_table.columns[:plast_attr]],
                      feat_table.loc[~mask, prop_feat_less_blank[prop_feat_less_blank].index]], axis=1)

In [86]:
filterBlank(df)

Found 3 samples containing "blank" in the name.
Found 974 features.
Found 32 features whose intensity was greater than 100.0% of the 
          average intensity of blank samples in 90.0% samples at least.


,filename,ATTRIBUTE_sample,ATTRIBUTE_Source,ATTRIBUTE_GasProduction24hmLg,ATTRIBUTE_GPcat,ATTRIBUTE_CH4ppm,ATTRIBUTE_CH424hmLg,ATTRIBUTE_CH4Ycat,ATTRIBUTE_TVFAcat,ATTRIBUTE_Acecat,...,605_0.22,625_0.23,657_0.24,705_0.25,799_0.28,841_0.29,915_0.32,988_0.34,1032_0.36,1035_0.37
0,1.mzML,Sample,Asparagopsis taxiformis,114.7,low,82.0,0.11,low,low,low,...,1.000000e-05,7.871836,0.524550,6.879348e-02,9.770000e-07,1.120000e-06,0.048014,3.567550e-01,0.022454,197.969717
1,10.mzML,Sample,Mastocarpus papillatus,130.5,norm,6825.0,10.65,norm,norm,norm,...,4.810000e-06,23.398683,0.002255,5.710000e-05,5.881010e-04,1.010000e-05,0.000000,7.996355e-01,0.000370,5.939669
2,11.mzML,Sample,Sargassum fluitans,131.4,norm,6751.0,10.60,norm,norm,norm,...,3.594680e-04,4.413216,11.377640,1.470000e-06,3.931943e-02,4.390000e-06,0.006904,9.640000e-09,0.000110,24.023134
3,12.mzML,Sample,Dictyopteris undulata,129.5,norm,7625.0,11.84,norm,norm,norm,...,3.054060e-04,1480.992375,16.609931,8.570000e-05,3.120000e-05,1.520000e-07,26801.172340,2.510962e-01,0.000000,1050.631061
4,13.mzML,Sample,Soleria tenera,126.4,norm,5849.0,13.86,norm,norm,norm,...,3.250000e-05,296.273668,35.907945,3.482598e+00,1.233860e-01,9.160000e-06,1.321999,6.648704e+00,0.010552,21.306560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,71.mzML,Sample,Mastocarpus jardinii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.928160e-04,0.125759,61.902010,3.608533e-03,9.700000e-08,9.270000e-07,0.000000,1.912280e-04,0.003509,531.443639
69,72.mzML,Sample,Pevetiopsis limitata,131.3,norm,7585.0,11.77,norm,norm,norm,...,3.110000e-05,0.094545,26.335223,8.250000e-05,3.750000e-07,2.833640e-04,0.003388,0.000000e+00,0.002485,14.273601
70,73.mzML,Sample,Stephanocystis neglecta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.030000e-07,10.744302,788.919450,2.872090e+01,1.350000e-05,2.346503e+01,0.029205,1.120000e-08,0.000271,26.018043
71,8.mzML,Sample,Pelvetiopsis limitata,131.3,norm,7585.0,11.77,norm,norm,norm,...,1.980000e-07,61.593691,11.819502,6.270000e-08,6.140000e-07,3.230000e-05,0.002922,5.989585e-03,4.606882,30.477288


In [88]:
filterBlank(df, prop_blank_feats=0.7, prop_samples=0.8)

Found 3 samples containing "blank" in the name.
Found 974 features.
Found 126 features whose intensity was greater than 70.0% of the 
          average intensity of blank samples in 80.0% samples at least.


,filename,ATTRIBUTE_sample,ATTRIBUTE_Source,ATTRIBUTE_GasProduction24hmLg,ATTRIBUTE_GPcat,ATTRIBUTE_CH4ppm,ATTRIBUTE_CH424hmLg,ATTRIBUTE_CH4Ycat,ATTRIBUTE_TVFAcat,ATTRIBUTE_Acecat,...,976_0.33,978_0.34,981_0.34,988_0.34,998_0.34,1027_0.36,1030_0.36,1032_0.36,1035_0.37,1037_0.37
0,1.mzML,Sample,Asparagopsis taxiformis,114.7,low,82.0,0.11,low,low,low,...,5.360000e-09,4.100000e-11,1.600000e-09,3.567550e-01,3.460000e-14,0.024402,0.353150,0.022454,197.969717,210.997868
1,10.mzML,Sample,Mastocarpus papillatus,130.5,norm,6825.0,10.65,norm,norm,norm,...,9.500000e-10,9.410000e-11,1.110000e-09,7.996355e-01,5.200000e-10,0.012347,0.100389,0.000370,5.939669,961.478403
2,11.mzML,Sample,Sargassum fluitans,131.4,norm,6751.0,10.60,norm,norm,norm,...,5.630000e-07,1.630000e-06,3.880000e-09,9.640000e-09,7.460000e-14,0.000000,0.000000,0.000110,24.023134,660.696140
3,12.mzML,Sample,Dictyopteris undulata,129.5,norm,7625.0,11.84,norm,norm,norm,...,0.000000e+00,0.000000e+00,1.600000e-06,2.510962e-01,3.190000e-12,0.010229,0.225636,0.000000,1050.631061,380.983806
4,13.mzML,Sample,Soleria tenera,126.4,norm,5849.0,13.86,norm,norm,norm,...,4.330000e-07,5.440000e-08,1.360000e-07,6.648704e+00,3.100000e-12,1.023163,2.927845,0.010552,21.306560,343.420992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,71.mzML,Sample,Mastocarpus jardinii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.940000e-07,1.040000e-08,6.298980e-03,1.912280e-04,3.610556e-01,0.000000,504.704233,0.003509,531.443639,354.418215
69,72.mzML,Sample,Pevetiopsis limitata,131.3,norm,7585.0,11.77,norm,norm,norm,...,2.360000e-06,3.510000e-11,2.140000e-08,0.000000e+00,7.610000e-10,0.608255,0.021185,0.002485,14.273601,8682.073987
70,73.mzML,Sample,Stephanocystis neglecta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.003990e-01,6.020000e-07,3.790000e-08,1.120000e-08,1.100000e-09,0.285080,1.749333,0.000271,26.018043,1656.547325
71,8.mzML,Sample,Pelvetiopsis limitata,131.3,norm,7585.0,11.77,norm,norm,norm,...,1.210000e-06,6.450000e-12,1.896280e-04,5.989585e-03,7.680000e-10,0.021263,0.012253,4.606882,30.477288,4826.239877


In [89]:
filtered = filterBlank(df, prop_blank_feats=0.7)
filtered.to_csv("MSHUB_feature_table_Metadata_combined_blank_subtracted_sample_vs_blank_for_MA_filtered.csv",
                index=None)

Found 3 samples containing "blank" in the name.
Found 974 features.
Found 41 features whose intensity was greater than 70.0% of the 
          average intensity of blank samples in 90.0% samples at least.
